Importing and Installing dependencies

In [124]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp 

Keypoints using mp_holistic and mp_drawing

In [125]:
mp_holist = mp.solutions.holistic 
mp_draw = mp.solutions.drawing_utils

In [126]:
def mediapipe_detection(img, model):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img.flags.writeable = False                 
    result = model.process(img)                 # Make prediction
    img.flags.writeable = True                   
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) 
    return img, result

In [127]:
def draw_landmarks(img, result):
    mp_draw.draw_landmarks(img, result.face_landmarks, mp_holist.FACEMESH_CONTOURS) # Draw face connections
    mp_draw.draw_landmarks(img, result.pose_landmarks, mp_holist.POSE_CONNECTIONS) # Draw pose connections
    mp_draw.draw_landmarks(img, result.left_hand_landmarks, mp_holist.HAND_CONNECTIONS) # Draw left hand connections
    mp_draw.draw_landmarks(img, result.right_hand_landmarks, mp_holist.HAND_CONNECTIONS) # Draw right hand connections

In [128]:
def draw_styled_landmarks(img, result):
    mp_draw.draw_landmarks(img, result.face_landmarks, mp_holist.FACEMESH_CONTOURS, 
                             mp_draw.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), # color the joint 
                             mp_draw.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1) #color the connection
                             ) 
    # mp_draw.draw_landmarks(img, result.face_landmarks, mp_holist.FACEMESH_CONTOURS, 
    #                          mp_draw.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), # color the joint 
    #                          mp_draw.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1) #color the connection
    #                          ) 
    
    mp_draw.draw_landmarks(img, result.pose_landmarks, mp_holist.POSE_CONNECTIONS,
                             mp_draw.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_draw.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    mp_draw.draw_landmarks(img, result.left_hand_landmarks, mp_holist.HAND_CONNECTIONS, 
                             mp_draw.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_draw.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    mp_draw.draw_landmarks(img, result.right_hand_landmarks, mp_holist.HAND_CONNECTIONS, 
                             mp_draw.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_draw.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [129]:
mp_holist.POSE_CONNECTIONS

frozenset({(0, 1),
           (0, 4),
           (1, 2),
           (2, 3),
           (3, 7),
           (4, 5),
           (5, 6),
           (6, 8),
           (9, 10),
           (11, 12),
           (11, 13),
           (11, 23),
           (12, 14),
           (12, 24),
           (13, 15),
           (14, 16),
           (15, 17),
           (15, 19),
           (15, 21),
           (16, 18),
           (16, 20),
           (16, 22),
           (17, 19),
           (18, 20),
           (23, 24),
           (23, 25),
           (24, 26),
           (25, 27),
           (26, 28),
           (27, 29),
           (27, 31),
           (28, 30),
           (28, 32),
           (29, 31),
           (30, 32)})

In [130]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holist.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        draw_styled_landmarks(image, results)

        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [131]:
cap.release()
cv2.destroyAllWindows()

Extracting keypoint values

In [132]:
results.pose_landmarks.landmark[0].visibility

0.9999426007270813

In [133]:
len(results.pose_landmarks.landmark)

33

In [134]:
def extract_keypoints(results):
    pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    left_hnd=np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    right_hnd=np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    face=np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    return np.concatenate([pose,left_hnd,right_hnd,face])
# concatenating for the model to detect the sign language

In [135]:
extract_keypoints(results).shape

(1662,)

Setting up folders for collection

In [136]:
# Path for exported data, numpy arrays
Model_Data=os.path.join('Data for different actions')

actions = np.array(['hello', 'thanks', 'iloveyou'])

no_of_seqs = 30

# 30 frames in length
seq_length = 30

In [137]:
# just creating the folders and sub folders

for action in actions: 
    for seq in range(no_of_seqs):
        try: 
            os.makedirs(os.path.join(Model_Data, action, str(seq)))
        except:
            pass

Collecting keypoint values for Training nd Testing

In [138]:
# Set mediapipe model 
cap = cv2.VideoCapture(0)

with mp_holist.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for seq in range(no_of_seqs):
            for frame_num in range(seq_length):

                ret, frame = cap.read() 
                img, results = mediapipe_detection(frame, holistic)
                # print(results)
                draw_styled_landmarks(img, results)
                


                # logic is for the formating portion
                if frame_num == 0: 
                    cv2.putText(img, 'DATA COLLECTION STARTED', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(img, 'Collecting frames for - {} and Sequence Number - {}'.format(action, seq), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Window', img)
                    # providing the break for adjusting the posture
                    cv2.waitKey(2000) #2 sec
                else: 
                    cv2.putText(img, 'Collecting frames for - {} and Sequence Number - {}'.format(action, seq), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Window', img)
                

                keypoints = extract_keypoints(results)
                npy_path = os.path.join(Model_Data, action, str(seq), str(frame_num))
                np.save(npy_path, keypoints)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [139]:
cap.release()
cv2.destroyAllWindows()

Preprocessing data and creating labels w.r.t. actions

In [140]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [141]:
labelMap = {label:num for num, label in enumerate(actions)}

In [142]:
labelMap

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [143]:
seqs, labels = [], []

for action in actions:
    for seq in range(no_of_seqs):
        window = []
        for frame_num in range(seq_length):
            res = np.load(os.path.join(Model_Data, action, str(seq), "{}.npy".format(frame_num))) 
            window.append(res)
        seqs.append(window)

        labels.append(labelMap[action])

In [144]:
np.array(seqs).shape

(90, 30, 1662)

In [145]:
X_data = np.array(seqs)

In [146]:
X_data.shape

(90, 30, 1662)

In [147]:
# changing the labels from 0,1,2 to categorical data for easier accessebility
Y_label = to_categorical(labels).astype(int)
Y_label

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

In [148]:
# splitting
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_label, test_size=0.05)
X_test.shape

(5, 30, 1662)

Building and training LSTM neural network

In [149]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [150]:
# adding the logs folder
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [151]:
# neural network

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))

# adding 64 units for dense layer
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

c:\Users\somya\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [152]:
# eg
eg_res = [.7, 0.2, 0.1]
actions[np.argmax(eg_res)]

'hello'

In [153]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [154]:
model.fit(X_train, Y_train, epochs=300, callbacks=[tb_callback])
# tensorboard --logdir=.

Epoch 1/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 76ms/step - categorical_accuracy: 0.3326 - loss: 1.9655
Epoch 2/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - categorical_accuracy: 0.3170 - loss: 1.5603
Epoch 3/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - categorical_accuracy: 0.2857 - loss: 3.2076
Epoch 4/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - categorical_accuracy: 0.3248 - loss: 1.1021
Epoch 5/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - categorical_accuracy: 0.4131 - loss: 1.0807
Epoch 6/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - categorical_accuracy: 0.4559 - loss: 1.1760
Epoch 7/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - categorical_accuracy: 0.5127 - loss: 1.5810
Epoch 8/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - categorical_accuracy: 0.3327 - loss: 2.7976
Epoch 9/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - categorical_accuracy: 0.4501 - loss: 2.5427
Epoch 10/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - categorical_accuracy: 0.1976 - loss: 1.1051
Epoch 11/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0

In [155]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                   │ (None, 30, 64)         │       442,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 30, 128)        │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,790,027 (6.83 MB)

 Trainable params: 596,675 (2.28 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,193,352 (4.55 MB)

8. Making the predictions

In [156]:
res=model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 700ms/step


In [157]:
# again the actions with the max value provided by softmax is returned
actions[np.argmax(res[0])]

'thanks'

In [158]:
actions[np.argmax(Y_test[4])]

'iloveyou'

Saving weights for future accessability

In [159]:
model.save('action.h5')

In [160]:
# del model

In [161]:
model.load_weights('action.h5')

Evaluation using Confusion Matrix and Accuracy score

In [162]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [163]:
Y_hat = model.predict(X_train)

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step 


In [164]:
Y_true = np.argmax(Y_train, axis=1).tolist()
# one hot encoding
Y_hat = np.argmax(Y_hat, axis=1).tolist()

In [165]:
Y_hat

[2,
 2,
 2,
 2,
 0,
 1,
 1,
 0,
 2,
 1,
 0,
 0,
 0,
 1,
 0,
 2,
 1,
 2,
 0,
 0,
 2,
 1,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 1,
 0,
 2,
 1,
 0,
 1,
 2,
 0,
 1,
 0,
 2,
 1,
 1,
 1,
 1,
 1,
 0,
 2,
 0,
 0,
 1,
 2,
 1,
 2,
 1,
 0,
 0,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 0,
 2,
 0,
 1,
 1,
 0,
 2,
 0,
 0,
 2,
 2,
 0,
 2,
 2,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1]

In [166]:
# confution matrix
multilabel_confusion_matrix(Y_true, Y_hat)

array([[[55,  0],
        [ 0, 30]],

       [[56,  0],
        [ 0, 29]],

       [[59,  0],
        [ 0, 26]]], dtype=int64)

In [167]:
accuracy_score(Y_true, Y_hat)

1.0

FINAL Testing in real time

In [168]:
# for coloring the actions
colours = [(245,117,16), (117,245,16), (16,117,245)]

def prob_vis(res, actions, input_frame, colours):
    
    output_frame = input_frame.copy()
    
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colours[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [171]:
sequence=[]
sentence=[]
prediction=[]
threshold=0.5

cap = cv2.VideoCapture(0)
# Set mediapipe mqodel 
with mp_holist.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        ret, frame = cap.read()

        img, result = mediapipe_detection(frame, holistic)
        print(result)
        
        draw_styled_landmarks(img, result)

        # ==> prediction logic
        keypoints=extract_keypoints(result)
        sequence.append(keypoints)
        # sequence.insert(0,keypoints)

        sequence=sequence[-30:]

        if len(sequence)==30:
            res=model.predict(np.expand_dims(sequence,axis=0))[0]
            print(actions[np.argmax(res)])
            prediction.append(np.argmax(res))

        # ==> visualization logic
            if np.unique(prediction[-10:])[0]==np.argmax(res):
                if res[np.argmax(res)] > threshold: 
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

                if len(sentence) > 5: 
                    sentence = sentence[-5:]



            # ==> Visualiz probabilities
            # entering the prob func for the better visualization
            img = prob_vis(res, actions, img, colours)



        cv2.rectangle(img, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(img, ' '.join(sentence), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)


        cv2.imshow('OpenCV Window', img)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [170]:
cap.release()
cv2.destroyAllWindows()